<a href="https://colab.research.google.com/github/jmtcabili/CEPARCO-Integrating-Project-ARIMA/blob/main/Autoressive_Coefficient_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile lagged.cu
#include <stdio.h>
#include <stdlib.h>

__global__
void autoregressive(size_t n, float *lagged, float *in, int lagged_cols)
{
    int row = blockIdx.y * blockDim.y + threadIdx.y; // Row index
    int col = blockIdx.x * blockDim.x + threadIdx.x; // Column index

    int rowStride = blockDim.y * gridDim.y;
    int colStride = blockDim.x * gridDim.x;

    for (int i = row; i < n; i+= rowStride){
      for (int j = col; j < lagged_cols; j+= colStride){
        if (j == 0){
          lagged[i * lagged_cols + j] = 1;
        }else if (i < n && j < lagged_cols) {
            int index = i - j;
            if (index < 0) {
                lagged[i * lagged_cols + j] = 0; // Assign zero for out-of-bounds indices
            } else {
                lagged[i * lagged_cols + j] = in[index]; // Assign lagged value
            }
        }
      }
    }

}

__global__
void transpose(float *out, float *in, int p, size_t ARRAY_SIZE){

  int col = blockIdx.x * blockDim.x + threadIdx.x;
  int row = blockIdx.y * blockDim.y + threadIdx.y;

  if (row < p && col < ARRAY_SIZE) {
      out[row * ARRAY_SIZE + col] = in[col * p + row];
  }

}

__global__
void matMulNaive(float *dest, float *in1, float *in2,
                 size_t in1_height, size_t in2_height,
                 size_t in1_width, size_t in2_width)
{

    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    // Each thread computes one element of the result matrix
    float cValue = 0;

    if (row < in1_height && col < in2_width) {
        // Matrix multiplication: in1 (lagged_cols x n) * in2 (n x lagged_cols)
        for (int k = 0; k < in1_width; ++k) {
            cValue += in1[row * in1_width + k] * in2[k * in2_width + col];
        }
        dest[row * in2_width + col] = cValue;
    }

    //p+1 n x n p+1 -> first mul = p+1 mat
    //p+1 p+1 x p+1 n -> second mul = p+1 x n
}


//Matrix inverse functions
__global__ void nodiag_normalize(float *A, float *I, int n, int i){
	int x = blockIdx.x * blockDim.x + threadIdx.x;
	int y = blockIdx.y * blockDim.y + threadIdx.y;

	if (x < n && y < n)
	if (x == i && x!=y){
		I[x*n + y] /= A[i*n + i];
		A[x*n + y] /= A[i*n + i];
	}

}

__global__ void diag_normalize(float *A, float *I, int n, int i){
	int x = blockIdx.x * blockDim.x + threadIdx.x;
	int y = blockIdx.y * blockDim.y + threadIdx.y;

	if (x < n && y < n)
	if (x == y && x == i){
		I[x*n + y] /= A[i*n + i];
		A[x*n + y] /= A[i*n + i];
	}

}

__global__ void gaussjordan(float *A, float *I, int n, int i)
{
	int x = blockIdx.x * blockDim.x + threadIdx.x;
	int y = blockIdx.y * blockDim.y + threadIdx.y;

	if (x < n && y < n){
		if (x != i){
			I[x*n + y] -= I[i*n + y] * A[x*n + i];
			if (y != i){
				A[x*n + y] -= A[i*n + y] * A[x*n + i];
			}
		}
	}

}

__global__ void set_zero(float *A, float *I, int n, int i){
	int x = blockIdx.x * blockDim.x + threadIdx.x;
	int y = blockIdx.y * blockDim.y + threadIdx.y;

	if (x < n && y < n){
		if (x != i){
			if (y == i){
				A[x*n + y] = 0;
			}
		}
	}
}


int main (){

  //dataset
  const size_t ARRAY_SIZE = 10;
  const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
  //const size_t numOfLoops = 30;


  //arima parameters and variables;

  const int p = 4;
  const int lagged_cols = p + 1;
  const int q = 1;

  const size_t AR_SIZE = p + 1;
  const size_t AR_BYTES = AR_SIZE * sizeof(float);
  const size_t X_BYTES = ARRAY_SIZE * lagged_cols * sizeof(float);
  const size_t PXP_BYTES = AR_SIZE * AR_SIZE * sizeof(float);

  // declare arrays
  float *in, *out, *lagged, *transposed, *transposed2,
        *prod1, *inverse, *identity, *prod2, *AR_COEFF;
  cudaMallocManaged(&in, ARRAY_BYTES);
  cudaMallocManaged(&out, ARRAY_BYTES);
  cudaMallocManaged(&lagged, X_BYTES); //same amount of rows and p cols
  cudaMallocManaged(&transposed,X_BYTES);
  cudaMallocManaged(&transposed2,X_BYTES);
  cudaMallocManaged(&prod2,X_BYTES);
  cudaMallocManaged(&prod1,PXP_BYTES);
  cudaMallocManaged(&inverse,PXP_BYTES);
  cudaMallocManaged(&identity,PXP_BYTES);
  cudaMallocManaged(&AR_COEFF, AR_BYTES);


  int device = -1;
  cudaGetDevice(&device);  //get GPU id

  // memory advise
  cudaMemAdvise(in, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
  cudaMemAdvise(in, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);


  cudaMemPrefetchAsync(in,ARRAY_BYTES,cudaCpuDeviceId,NULL);                       //"prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(lagged,ARRAY_SIZE * lagged_cols * sizeof(float),device,NULL);         //"prefetch data" to create GPU page memory

  for (size_t i=0; i<ARRAY_SIZE; i++)
      in[i] = i % 5 + 1.0;

  cudaMemPrefetchAsync(in,ARRAY_BYTES,device,NULL);                                //prefetch from CPU to GPU

  dim3 threadsPerBlock(16, 16);
  dim3 numBlocks((ARRAY_SIZE + threadsPerBlock.x-1)/threadsPerBlock.x,
                 (lagged_cols + threadsPerBlock.y-1)/threadsPerBlock.y);

  autoregressive<<<numBlocks, threadsPerBlock>>> (ARRAY_SIZE,lagged, in, lagged_cols);
  cudaGetLastError();

  // synchronize GPU with CPU
  cudaDeviceSynchronize();

  // prefetch from GPU to CPU
  cudaMemPrefetchAsync(lagged,ARRAY_SIZE * lagged_cols * sizeof(float),cudaCpuDeviceId,NULL);


  for (int i = ARRAY_SIZE-10; i < ARRAY_SIZE; i++){
    for (int j = 0; j < lagged_cols; j++){
      printf("%.2f ", lagged[i*lagged_cols+j]);
    }
    printf("\n");
  }



  //---------- Transposing lagged matrix ----------------//
  //try removing later along with prev prefetch async and print


  cudaMemAdvise(lagged, X_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
  cudaMemAdvise(lagged, X_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);


  cudaMemPrefetchAsync(lagged,X_BYTES,cudaCpuDeviceId,NULL);                       //"prefetch data" to create CPU page memory

  cudaMemPrefetchAsync(transposed,X_BYTES, device, NULL);                                   //"prefetch data" to create GPU page memory

  cudaMemPrefetchAsync(lagged, X_BYTES, device, NULL);


  //dim3 dimGrid(ARRAY_SIZE/TILE_DIM, p/TILE_DIM, 1);
  //dim3 dimBlock(TILE_DIM, BLOCK_ROWS, 1);

  transpose<<<numBlocks, threadsPerBlock>>>(transposed, lagged, lagged_cols, ARRAY_SIZE);
  cudaGetLastError();

  // synchronize GPU with CPU
  cudaDeviceSynchronize();

  // prefetch from GPU to CPU
  cudaMemPrefetchAsync(transposed,X_BYTES,cudaCpuDeviceId,NULL);


  //---printing tranposed---//
  printf("\n");

  for (int i = 0; i < lagged_cols; i++){
    for (int j = 0; j < 10; j++){
      printf("%.2f ", transposed[i*ARRAY_SIZE+j]);
    }
    printf("\n");
  }

  cudaMemPrefetchAsync(prod1, PXP_BYTES, device, NULL);
  cudaMemPrefetchAsync(lagged, X_BYTES, device, NULL);
  cudaMemPrefetchAsync(transposed, X_BYTES, device, NULL);

  matMulNaive<<<numBlocks, threadsPerBlock>>>(prod1, transposed, lagged,
                                              lagged_cols, ARRAY_SIZE, ARRAY_SIZE, lagged_cols);

  cudaDeviceSynchronize();
  cudaError_t err = cudaGetLastError();
  if (err != cudaSuccess) {
      printf("CUDA Error: %s\n", cudaGetErrorString(err));
  }

  // Prefetch result back to CPU
  cudaMemPrefetchAsync(prod1, PXP_BYTES, cudaCpuDeviceId, NULL);

  // Print results if needed
  printf("\nMatrix multiplication result:\n");
  for (int i = 0; i < lagged_cols; i++) {
      for (int j = 0; j < lagged_cols; j++) {
          printf("%.2f ", prod1[i * lagged_cols + j]);
      }
      printf("\n");
  }

  dim3 threadsPerBlockInv(lagged_cols, lagged_cols);
  dim3 numBlocksInv((lagged_cols + lagged_cols -1) / lagged_cols,
                 (lagged_cols+lagged_cols-1)/lagged_cols);

  cudaMemPrefetchAsync(inverse, PXP_BYTES, cudaCpuDeviceId, NULL);
  cudaMemPrefetchAsync(prod1, PXP_BYTES, device, NULL);

  //set identity matrix
  for (int i = 0; i < lagged_cols; i++){
    for (int j = 0; j < lagged_cols; j++){
      if (i == j)
        inverse[i * lagged_cols + j] = 1.0;
      else
        inverse[i * lagged_cols + j] = 0.0;
    }
  }

  printf("\nIdentity Matrix result:\n");
  for (int i = 0; i < lagged_cols; i++) {
      for (int j = 0; j < lagged_cols; j++) {
          printf("%.2f ", inverse[i * lagged_cols + j]);
      }
      printf("\n");
  }


  for (int i = 0; i < lagged_cols; i++){
    nodiag_normalize <<<numBlocksInv, threadsPerBlockInv >>>(prod1, inverse, lagged_cols, i);
    cudaDeviceSynchronize();
		diag_normalize <<<numBlocksInv, threadsPerBlockInv>>>(prod1, inverse, lagged_cols, i);
    cudaDeviceSynchronize();
		gaussjordan <<<numBlocksInv, threadsPerBlockInv>>>(prod1, inverse,lagged_cols, i);
    cudaDeviceSynchronize();
		set_zero <<<numBlocksInv, threadsPerBlockInv>>>(prod1, inverse, lagged_cols, i);
    cudaDeviceSynchronize();
  }

  // Prefetch result back to CPU
  cudaMemPrefetchAsync(inverse, PXP_BYTES, cudaCpuDeviceId, NULL);

  printf("\nInverse result:\n");
  for (int i = 0; i < lagged_cols; i++) {
      for (int j = 0; j < lagged_cols; j++) {
          printf("%.9f ", inverse[i * lagged_cols + j]);
      }
      printf("\n");
  }

  cudaMemPrefetchAsync(transposed, X_BYTES, device, NULL);
  cudaMemPrefetchAsync(inverse, PXP_BYTES, device, NULL);
  cudaMemPrefetchAsync(prod2, X_BYTES, device, NULL);

  matMulNaive<<<numBlocks, threadsPerBlock>>>(prod2, inverse, transposed, lagged_cols, lagged_cols, lagged_cols, ARRAY_SIZE);

  cudaDeviceSynchronize();
  err = cudaGetLastError();
  if (err != cudaSuccess) {
      printf("CUDA Error: %s\n", cudaGetErrorString(err));
  }

  // Prefetch result back to CPU
  cudaMemPrefetchAsync(prod2, X_BYTES, cudaCpuDeviceId, NULL);

  printf("\n");
  // Print results if needed
  for (int i = 0; i < lagged_cols; i++){
    for (int j = 0; j < 10; j++){
      printf("%.9f ", prod2[i*ARRAY_SIZE+j]);
    }
    printf("\n");
  }

  cudaMemPrefetchAsync(prod2, X_BYTES, device, NULL);
  cudaMemPrefetchAsync(AR_COEFF, AR_BYTES, device, NULL);

  matMulNaive<<<numBlocks, threadsPerBlock>>>(AR_COEFF, prod2, in, lagged_cols, ARRAY_SIZE, ARRAY_SIZE, 1);

  cudaDeviceSynchronize();
  err = cudaGetLastError();
  if (err != cudaSuccess) {
      printf("CUDA Error: %s\n", cudaGetErrorString(err));
  }

  // Prefetch result back to CPU
  cudaMemPrefetchAsync(AR_COEFF, X_BYTES, cudaCpuDeviceId, NULL);

  for (int i = 0; i < lagged_cols; i++){
    printf("%.5f\n", AR_COEFF[i]);
  }






  //cudaDeviceSynchronize();


  //free memory
  cudaFree(in);
  cudaFree(out);
  cudaFree(lagged);
  cudaFree(transposed);
  cudaFree(prod1);
  cudaFree(prod2);
  cudaFree(inverse);


  return 0;
}


Overwriting lagged.cu


In [ ]:
%%shell
nvcc -arch=sm_75 lagged.cu -o lagged

lagged.cu(134): warning #177-D: variable "q" was declared but never referenced
    const int q = 1;
              ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"



In [ ]:
%%shell
nvprof ./lagged

==6122== NVPROF is profiling process 6122, command: ./lagged
1.00 0.00 0.00 0.00 0.00 
1.00 1.00 0.00 0.00 0.00 
1.00 2.00 1.00 0.00 0.00 
1.00 3.00 2.00 1.00 0.00 
1.00 4.00 3.00 2.00 1.00 
1.00 5.00 4.00 3.00 2.00 
1.00 1.00 5.00 4.00 3.00 
1.00 2.00 1.00 5.00 4.00 
1.00 3.00 2.00 1.00 5.00 
1.00 4.00 3.00 2.00 1.00 

1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 
0.00 1.00 2.00 3.00 4.00 5.00 1.00 2.00 3.00 4.00 
0.00 0.00 1.00 2.00 3.00 4.00 5.00 1.00 2.00 3.00 
0.00 0.00 0.00 1.00 2.00 3.00 4.00 5.00 1.00 2.00 
0.00 0.00 0.00 0.00 1.00 2.00 3.00 4.00 5.00 1.00 

Matrix multiplication result:
10.00 25.00 21.00 18.00 16.00 
25.00 85.00 65.00 51.00 44.00 
21.00 65.00 69.00 53.00 43.00 
18.00 51.00 53.00 60.00 47.00 
16.00 44.00 43.00 47.00 56.00 

Identity Matrix result:
1.00 0.00 0.00 0.00 0.00 
0.00 1.00 0.00 0.00 0.00 
0.00 0.00 1.00 0.00 0.00 
0.00 0.00 0.00 1.00 0.00 
0.00 0.00 0.00 0.00 1.00 

Inverse result:
0.439719111 -0.092227302 -0.016625991 -0.020980148 -0.022794995 
